In [29]:
import cv2
import numpy as np
import redis
from insightface.app import FaceAnalysis

In [21]:
# Configure face analysis
faceapp = FaceAnalysis(name='buffalo_sc',
                       root='insightface_model',
                       providers=['CPUExecutionProvider'])

faceapp.prepare(ctx_id=0, det_size=(640, 640), det_thresh=0.5)
# Warning: don’t set det_thresh < 0.3

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_sc/det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_sc/w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


### Step 1: Collect Person and Role

In [22]:
# Python input
person_name = input('Enter your name: ')

trials = 3
for i in range(trials):
    role = input("""
    Please choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
    """)

    if role in ('1', '2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        break
    else:
        print('Invalid entry, try again')
        if i == 2:
            print('Exceeds maximum trials')

key = person_name + '@' + role
print('Your name =', person_name)
print('Your role =', role)
print('Key =', key)

Your name = malani
Your role = Teacher
Key = malani@Teacher


### Step 2: Collect Samples

In [23]:
cap = cv2.VideoCapture(0) # default camera, 1 external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
    
    # get results from insightface model
    results = faceapp.get(frame, max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 1)

        # facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)

    if sample >= 200:
        break

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'): # this is triggered only when I press letter q (Lowercase q) in my keyboard
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [24]:
len(face_embeddings)

200

### Step 3: Optimize data and save in Redis database

In [25]:
# Compute the Mean of Facial Embeddings
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [26]:
# Convert the Mean Embedding to Bytes
x_mean_bytes = x_mean.tobytes()

In [30]:
# Connect to redis client
hostname = 'redis-12955.crce179.ap-south-1-1.ec2.redns.redis-cloud.com'
portnumber = 12955
password = '0AXzSpM3wa8utdJhuvvpRF8Iwc4TNVT3'

r = redis.StrictRedis(host=hostname, port=portnumber, password=password)

In [31]:
# Store in redis
r.hset(name="test:register", key=key, value=x_mean_bytes)

1